In [3]:
!pip install python-bioformats javabridge

  Using cached python_bioformats-4.0.7-py3-none-any.whl (40.6 MB)
  Using cached javabridge-1.0.19.tar.gz (1.3 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import bioformats
import javabridge
import numpy as np
from PIL import Image
import os

ModuleNotFoundError: ignored

In [ ]:
# Initialize Java bridge
javabridge.start_vm(class_path=bioformats.JARS)

# Path to your CZI file
czi_file_path = 'your_file.czi'

# Open the CZI file
reader = bioformats.ImageReader(czi_file_path)

# Define the size of the subparts
subpart_width = 800
subpart_height = 600




In [ ]:
# Get the dimensions of the CZI file
image_width = reader.getSizeX()
image_height = reader.getSizeY()
num_channels = reader.getSizeC()
num_zstacks = reader.getSizeZ()

# Iterate through the CZI file to read and save subparts
for channel in range(num_channels):
    for zstack in range(num_zstacks):
        for x in range(0, image_width, subpart_width):
            for y in range(0, image_height, subpart_height):
                # Read the subpart
                subpart = reader.read(x=x, y=y, c=channel, z=zstack, width=subpart_width, height=subpart_height)

                # Convert subpart to a numpy array
                subpart_array = np.array(subpart, dtype=np.uint16)  # Adjust dtype as needed

                # Create a PIL Image from the numpy array
                subpart_image = Image.fromarray(subpart_array)

                # Define the output directory and filename
                output_dir = 'output/'
                os.makedirs(output_dir, exist_ok=True)
                output_filename = f'channel_{channel}_zstack_{zstack}_x_{x}_y_{y}.tif'
                output_path = os.path.join(output_dir, output_filename)

                # Save the subpart as a TIFF file
                subpart_image.save(output_path)



In [ ]:
# Clean up the Java bridge
javabridge.kill_vm()